In [1]:
// 1. Installation

//install libraries
import org.archive.archivespark._
import org.archive.archivespark.functions._
import org.archive.archivespark.specific.warc._

// data files - generic path from docker run -v
val cdxPath = "/data/arc_cdx/*.cdx"
val warcPath = "/data/warc"

import org.apache.spark.sql.{Row, SparkSession}

val session = spark.newSession

// collect all records

val r = ArchiveSpark.load(WarcSpec.fromFiles(cdxPath, warcPath))

In [2]:
val researchYear = "1999"

In [3]:
// 2. Count ALL objects from research Year
// 2.1 get data for text/html & HTTP 200 for each Year
val t1 = r.filter(r => r.timestamp.startsWith(researchYear))


In [4]:
// 3. generate basic data frame

val m = t1.map(f=> (f.originalUrl,f.digest,f.status,f.mime))
val df = session.createDataFrame(m).toDF("originalUrl","digest","status","mime").cache()

In [5]:
import org.apache.spark.sql.functions.count
import org.apache.spark.sql.functions._


In [6]:
df.filter(df("mime") === "text/html").select(countDistinct("originalUrl")).show()

+---------------------------+
|count(DISTINCT originalUrl)|
+---------------------------+
|                    1312057|
+---------------------------+



In [7]:
df.select(countDistinct("originalUrl")).show()

+---------------------------+
|count(DISTINCT originalUrl)|
+---------------------------+
|                    1454305|
+---------------------------+



In [8]:
df.groupBy("mime").agg(countDistinct("originalUrl") as "URLs").orderBy(desc("URLs")).show(300,false)

+---------------+-------+
|mime           |URLs   |
+---------------+-------+
|text/html      |1312057|
|im             |69634  |
|text/pl        |50052  |
|unk            |31296  |
|text/plain     |2117   |
|text/x         |155    |
|text/css       |118    |
|x              |63     |
|text/          |24     |
|text/nt-t      |18     |
|multip         |18     |
|text/c         |17     |
|text/rtf       |17     |
|text/nt-l      |11     |
|m              |9      |
|video/mpeg     |7      |
|model/vrml     |7      |
|test/pl        |5      |
|unknown        |3      |
|-              |3      |
|video/x        |3      |
|httpd/send     |2      |
|mess           |2      |
|video/quicktime|2      |
|text/xml       |1      |
|tpl            |1      |
|text/j         |1      |
|www/unknown    |1      |
|text/t         |1      |
|text/pt        |1      |
+---------------+-------+



In [9]:
df.groupBy("mime").agg(countDistinct("originalUrl") as "URLs").count()

30